In [1]:
#Импортируем библиотеки
import math as m
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import warnings
warnings.filterwarnings("ignore")

#Задаём функцию для аппроксимации
def mapping(x, a, b): 
    return a*x+b

In [2]:
!pip install vtk
import vtk

In [23]:
#Код для моделирования погонного затухания
# Класс расчётной сетки
class CalcMesh:

    # Конструктор сетки size x size точек с шагом h по пространству
    def __init__(self, size, step):
        # 2D-сетка из расчётных точек, у каждой из которых, тем не менее, 3 координаты
        self.nodes = np.mgrid[0:size-1:np.complex128(size), 0:0.5:np.complex128(size)]
        a = np.zeros((size,size))
        self.nodes *= step
        self.nodes = np.append(self.nodes, [np.zeros(shape=(size, size), dtype=np.double)], 0)

        # Модельная скалярная величина распределена как-то вот так
        self.smth = a

        # Профиль скорости взят какой-нибудь с потолка, есть только компонента по Z
        self.velocity = np.zeros(shape=(3, size, size), dtype=np.double)
        self.velocity[2] = np.power(self.nodes[0] - self.nodes[1], 2)

    # Метод отвечает за выполнение для всей сетки шага по времени величиной tau
    def move(self, tau, ru):
        # По сути метод просто двигает все точки c их текущими скоростями
        self.smth[ru] = 100 + self.smth[ru]- tau*ru
            

    # Метод отвечает за запись текущего состояния сетки в снапшот в формате VTK
    def snapshot(self, snap_number):
        # Сетка в терминах VTK
        structuredGrid = vtk.vtkStructuredGrid()
        # Точки сетки в терминах VTK
        points = vtk.vtkPoints()

        # Скалярное поле на точках сетки
        smth = vtk.vtkDoubleArray()
        smth.SetName("smth")

        # Векторное поле на точках сетки
        vel = vtk.vtkDoubleArray()
        vel.SetNumberOfComponents(3)
        vel.SetName("vel")

        # Обходим все точки нашей расчётной сетки
        # Делаем это максимально неэффективным, зато наглядным образом
        number = len(self.nodes[0])
        for i in range(0, number):
            for j in range(0, number):
                # Вставляем новую точку в сетку VTK-снапшота
                points.InsertNextPoint(self.nodes[0][i,j], self.nodes[1][i,j], self.nodes[2][i,j])
                # Добавляем значение скалярного поля в этой точке
                smth.InsertNextValue(self.smth[i,j])
                # Добавляем значение векторного поля в этой точке
                vel.InsertNextTuple((self.velocity[0][i,j], self.velocity[1][i,j], self.velocity[2][i,j]))

        # Задаём размеры VTK-сетки (в точках, по трём осям)
        structuredGrid.SetDimensions(number, number, 1)
        # Грузим точки в сетку
        structuredGrid.SetPoints(points)

        # Присоединяем векторное и скалярное поля к точкам
        structuredGrid.GetPointData().AddArray(smth)
        structuredGrid.GetPointData().AddArray(vel)

        # Создаём снапшот в файле с заданным именем
        writer = vtk.vtkXMLStructuredGridWriter()
        writer.SetInputDataObject(structuredGrid)
        writer.SetFileName("pogonoeZatuhanie2-" + str(snap_number) + ".vts")
        writer.Write()
    
        
print("all right")

all right


In [24]:
# Размер расчётной сетки, длина оптоволокна
flag = 1
size = int(input("Введите длину оптоволокна [км]: "))
l = int(input("Выберете рабочую длину волны [1310 // 1550 нм] : "))
# Шаг точек по пространству
h = 1
# Тип волокна
t = str(input("Выберете тип волокна [SMF // PSCF]: "))
# Определение погонного затухания

if l == 1310:
    if t == "SMF":
        tau = 0.33
    elif t == "PSCF":
        tau = 0.3
    else:
        print("Простите мы ещё не умеем моделировать данный тип волокна /n Введите один из предложенных типов: SMF или PSCF")
elif l == 1550:
    if t == "SMF":
        tau = 0.2
    elif t == "PSCF":
        tau = 0.15
    else:
        print("Простите мы ещё не умеем моделировать данный тип волокна \n Введите один из предложенных типов: SMF или PSCF")
        flag = 0

else:
    print("Простите мы ещё не умеем моделировать оптоволокно на данной длине волны \n Введите один из предложенных типов: 1310 или 1550 нм")
    flag = 0


if flag == 1:
    ru = 0
    # Создаём сетку заданного размера
    m = CalcMesh(size, h)
    # Пишем её начальное состояние в VTK
    m.snapshot(0)
    # Делаем шаги по времени,
    # на каждом шаге считаем новое состояние и пишем его в VTK
    for i in range(1, size+1):
        m.move(tau, ru)
        m.snapshot(i)
        ru += 1

    print("Ready")
    print("Запустите в Para View файл pogonoeZatuhanie2.vts")
    
    
#Проблема: ParaView запоминает максимальное число сцен, поэтому файл лучше всегда переименовывать для точности расчёта!

Введите длину оптоволокна [км]: 20
Выберете рабочую длину волны [1310 // 1550 нм] : 1550
Выберете тип волокна [SMF // PSCF]: PSCF
Ready
Запустите в Para View файл pogonoeZatuhanie2.vts
